In [1]:
# -*- coding: utf-8 -*-
# Adapted from lstm_text_generation.py in keras/examples

In [2]:
# imports

from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import numpy as np 
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
import nltk
from urllib import request
import requests
from bs4 import BeautifulSoup, NavigableString
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300


2022-10-05 17:14:37.922439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [34]:
# prepairing data 
print('Prepairing data...\n')
url = 'https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt'
text = request.urlopen(url).read().decode('utf8')

# regex cleaning 
pattern1 = re.compile(r'[*\[\]’\'\"`\)\(\--]')
pattern2 = re.compile(r'\s{1,}')
text = re.sub(pattern1, '', text)
text = re.sub(pattern2, ' ', text)

word_tokens = nltk.word_tokenize(text.lower())[0:5000] # is a list with all the words from a text 
# print(word_tokens)
# print(sentense_tokens)
# print(text)

Prepairing data...



In [4]:
# # frequency of words without removing stop words

# freq = nltk.FreqDist(word_tokens)

# plt.xticks(fontsize=6, rotation=90)
# plt.grid()
# plt.title('Frequency of some words', size=15)
# freq.plot(80,cumulative=False, show=False)
# plt.savefig('images/word_freq_with_stop_words.png', bbox_inches = "tight")

In [5]:
# more filtering

print('Filtering data...\n')
# removing stop words
# stop_words = set(nltk.corpus.stopwords.words("english"))
# stop_words.update([",", ".", ":", ";", "?", "!", '*', '--', '[', ']', '``', '\"\"', "\'\'"])
# word_tokens = [token for token in word_tokens if token not in stop_words] # words without stop words
# print(word_tokens)


# # lematization
# lemmatizer = nltk.WordNetLemmatizer()
# word_tokens = [lemmatizer.lemmatize(token.lower()) for token in word_tokens]
# # print(word_tokens)

Filtering data...



In [6]:
# # frequency of words with stop words removing

# freq = nltk.FreqDist(word_tokens)
# plt.xticks(fontsize=6, rotation=90)
# plt.grid()
# plt.title('Frequency of some words', size=15)
# freq.plot(80,cumulative=False, show=False)
# plt.savefig('images/word_freq_without_stop_words.png', bbox_inches = "tight")

In [7]:
# tags meaning parser

dict_tags_meaning = dict()
url = 'https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html'
response = requests.get(url, 'html_parser')
soup = BeautifulSoup(response.content)
for child in list(soup.table.children)[2:]:
    if isinstance(child, NavigableString):
        continue
    dict_tags_meaning[child.find_all('td')[1].text.strip()] = child.find_all('td')[2].text.strip()
# print(dict_tags_meaning)

In [8]:
# tags analyzis

# df_tags = pd.DataFrame(nltk.pos_tag(word_tokens), columns=['Word', 'Tag'])

# ser_tags_grouping = df_tags['Tag'].value_counts()
# ser_tags_grouping.rename(index=dict_tags_meaning, inplace=True)
# # print(ser_tags_grouping)
# plt.xticks(fontsize=10, rotation=90)
# plt.grid()
# plt.title('Frequency of tags', size=15)
# plt.plot(ser_tags_grouping)
# plt.savefig('images/tags_freq.png', bbox_inches = "tight")





In [9]:
# tokenizing data

print('Tokenizing data...\n')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(word_tokens)
total_words = len(tokenizer.word_index) + 1
# tokenizer.word_index # is a dictionary with unique words and their numeric values


Tokenizing data...



In [10]:
text = ' '.join(word_tokens).replace(' ,', ',').replace(' .', '.').replace(' :', ':').replace(' ?', '?').replace(' ;', ';').replace('wa', 'was')
# print(text)
sentense_tokens = nltk.sent_tokenize(text) # is a list with all the sentenses from a text
input_sequences = []
for line in sentense_tokens:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]

        input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = to_categorical(labels, num_classes=total_words)

# pd.DataFrame(input_sequences).to_csv('test.csv')

In [11]:
# # creating lookup tables

# print('Creating lookup tables...\n')
# set_chars = set([c for c in text])
# nb_chars = len(chars)
# char2index = dict((c, i) for i, c in enumerate(set_chars))
# index2char = dict((i, c) for i, c in enumerate(set_chars))

In [12]:

# print("Creating input and label text...")
# SEQLEN = 10
# STEP = 1

# input_chars = []
# label_chars = []
# for i in range(0, len(text) - SEQLEN, STEP):
#     input_chars.append(text[i:i + SEQLEN])
#     label_chars.append(text[i + SEQLEN])

In [13]:
# # vectorize the input and label chars
# # Each row of the input is represented by seqlen characters, each
# # represented as a 1-hot encoding of size len(char). There are
# # len(input_chars) such rows, so shape(X) is (len(input_chars),
# # seqlen, nb_chars).
# # Each row of output is a single character, also represented as a
# # dense encoding of size len(char). Hence shape(y) is (len(input_chars),
# # nb_chars).

# print("Vectorizing input and label text...")
# X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
# y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
# for i, input_char in enumerate(input_chars):
#     for j, ch in enumerate(input_char):
#         X[i, j, char2index[ch]] = 1
#     y[i, char2index[label_chars[i]]] = 1
# np.zeros(12)


In [14]:
# # # Build the model. We use a single RNN with a fully connected layer
# # # to compute the most likely predicted output char
# HIDDEN_SIZE = 128
# BATCH_SIZE = 128
# NUM_ITERATIONS = 25
# NUM_EPOCHS_PER_ITERATION = 1
# NUM_PREDS_PER_EPOCH = 100

# model = Sequential()
# model.add(SimpleRNN(500, return_sequences=False, input_shape=(max_sequence_len-1, 1), unroll=True))
# model.add(Dense(total_words))
# model.add(Activation("softmax"))

# model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
# history = model.fit(xs, ys, epochs=30, verbose=1)


In [15]:
# # We train the model in batches and test output generated at each step
# for iteration in range(NUM_ITERATIONS):
#     print("=" * 50)
#     print("Iteration #: %d" % (iteration))
#     model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

#     # testing model
#     # randoml       y choose a row from input_chars, then use it to
#     # generate text from model for next 100 chars
#     test_idx = np.random.randint(len(input_chars))
#     test_chars = input_chars[test_idx]
#     print("Generating from seed: %s" % (test_chars))
#     print(test_chars, end="")
#     for i in range(NUM_PREDS_PER_EPOCH):
#         Xtest = np.zeros((1, SEQLEN, nb_chars))
#         for i, ch in enumerate(test_chars):
#             Xtest[0, i, char2index[ch]] = 1
#         pred = model.predict(Xtest, verbose=0)[0]
#         ypred = index2char[np.argmax(pred)]
#         print(ypred, end="")
#         # move forward with test_chars + ypred
#         test_chars = test_chars[1:] + ypred
#     print()
total_words

65

In [16]:
model = Sequential()
model.add(Embedding(total_words, total_words/2, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = False)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=15, verbose=1)



2022-10-05 17:14:51.610499: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/15
3/3 [==============================] - 4s 161ms/step - loss: 4.1937 - accuracy: 0.0225
Epoch 2/15
3/3 [==============================] - 0s 150ms/step - loss: 4.1241 - accuracy: 0.0449
Epoch 3/15
3/3 [==============================] - 0s 152ms/step - loss: 4.0240 - accuracy: 0.0562
Epoch 4/15
3/3 [==============================] - 0s 156ms/step - loss: 3.7260 - accuracy: 0.1011
Epoch 5/15
3/3 [==============================] - 0s 151ms/step - loss: 3.3617 - accuracy: 0.1236
Epoch 6/15
3/3 [==============================] - 0s 151ms/step - loss: 2.8868 - accuracy: 0.1236
Epoch 7/15
3/3 [==============================] - 0s 148ms/step - loss: 2.4521 - accuracy: 0.3034
Epoch 8/15
3/3 [==============================] - 0s 149ms/step - loss: 2.2870 - accuracy: 0.2584
Epoch 9/15
3/3 [==============================] - 0s 148ms/step - loss: 1.9796 - accuracy: 0.3258
Epoch 10/15
3/3 [==============================] - 0s 161ms/step - loss: 1.7846 - accuracy: 0.3708
Epoch 11/15
3/3 [==

In [17]:
# acc = history.history['accuracy']
# loss = history.history['loss']
# epochs = range(len(acc))
# plt.plot(epochs, acc)
# plt.title('Training accuracy')
# plt.savefig('images/model_acc.png', bbox_inches='tight')
# plt.plot(epochs, loss, 'b')
# plt.title('Training loss')

# plt.savefig('images/model_loss.png')



In [33]:
seed_text = "down the"
next_words = 20
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)

    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break 
    seed_text += " " + output_word
print(seed_text)

down the wonderland adventures in wonderland lewis the millennium fulcrum edition 3 0 chapter down the rabbithole alice beginning to get tired
